<a href="https://colab.research.google.com/github/megazhuk/pycopy-build-in-colab/blob/main/pycopy_compile.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Init Workspace
git_pycopy = "/content/git_pycopy"
git_espidf = "/content/git_espidf"
dir_esp32 = git_pycopy + "/ports/esp32"
dir_mpy_cross = git_pycopy + '/mpy-cross'

# Downloads (git, modules)

In [ ]:
#@title GIT clone 'pfalcon/pycopy.git'
!git clone https://github.com/pfalcon/pycopy.git $git_pycopy

In [ ]:
#@title GIT clone 'espressif/esp-idf.git'
! git clone https://github.com/espressif/esp-idf.git $git_espidf


In [ ]:
#@title Choose esp-idf Version { run: "auto" }
ESP_IDF_version = "V4" #@param ["V4", "V3"]
var_to_find = 'ESPIDF_SUPHASH_' + ESP_IDF_version
separator = ' := '
tmp = 'not empty'
with open(dir_esp32+'/Makefile', 'r') as f:
  while(tmp != ''):
    tmp = f.readline()
    if len(tmp) == 0 : break
    if var_to_find in tmp:
      esp_sha = tmp.split(sep=separator, maxsplit=1)[-1][:-1]
      if len(esp_sha) == 40:
        print(f'Found: {esp_sha}')
        break
  if tmp == '':
    print (f'Hash not found by variable: {var_to_find}')

In [ ]:
#@title Apply esp-idf Version and get Submodules to it
! cd $git_espidf && git reset --hard $esp_sha
! cd $git_espidf && git submodule update --init --recursive

In [ ]:
#@title Install esp-idf (compiler, env, packages)
! cd $git_espidf && ./install.sh

In [ ]:
#@title Patch ESP-IDF's 'mbedtls' from 'pfalcon/mbedtls/'
dir_mbed = git_espidf + '/components/mbedtls/'
! cd $dir_mbed && rm -rf mbedtls && git clone -b mbedtls-2.16.5-idf-pycopy https://github.com/pfalcon/mbedtls/

# mpy-cross

In [ ]:
#@title Compile 'mpy-cross'
! cd $dir_mpy_cross && make

In [ ]:
#@title download 'mpy-cross', if you need it local
import ipywidgets as widgets
from IPython.display import display
from google.colab import files

button = widgets.Button(
    description="Get mpy-cross", 
    tooltip='Click to download',
    icon='file-download', # (FontAwesome names without the `fa-` prefix)
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    disabled=False,
    )

def on_button_clicked(b):
  # Display the message within the output widget.
  files.download(dir_mpy_cross+'/pycopy-cross')

button.on_click(on_button_clicked)
display(button)

# Build Pycopy Firmware

In [ ]:
#@title Configure ESP32 fw. NOT READY, YOU CAN SKIP FURTHER { run: "auto" }
board = "GENERIC" #@param ["GENERIC"] {allow-input: true}
flash_size = "4MB" #@param ["2MB", "4MB"] {allow-input: true}
flash_mode = "dio" #@param ["dio", "qio"]
#@markdown >(*flash_mode for ESP-WROOM-32 should be 'dio'*)
serial_port = "/dev/ttyUSB0" #@param {type:"string"}
BLE_enalbe = True #@param {type:"boolean"}
nimBLE_enable = True #@param {type:"boolean"}
nimBLE_only_bind = True #@param {type:"boolean"}
#@markdown >(*`only_bind` - to use own ESP32 lib instead full NimBLE*)

#@markdown TODO: SPIRAM enabler<br>
#@markdown TODO: understand BLE/BT/NIMBLE options

if not BLE_enalbe:
  nimBLE_enable = nimBLE_only_bind = False
content = (
# f'ESPIDF ?= {git_espidf}\n'
f'BOARD ?= {board}\n'
f'#PORT ?= {serial_port}\n'
f'#FLASH_MODE ?= {flash_mode}\n'
f'#FLASH_SIZE ?= {flash_size}\n'
f'MICROPY_PY_BLUETOOTH ?= {1 if BLE_enalbe else 0} \n'
f'MICROPY_BLUETOOTH_NIMBLE ?= {1 if nimBLE_enable else 0}\n'
f'MICROPY_BLUETOOTH_NIMBLE_BINDINGS_ONLY = {1 if nimBLE_only_bind else 0}\n'
f'include Makefile'
)

# makefile = 'makefile'
# f = open(dir_esp32+'/'+makefile,"w")
# f.write (content)
# f.close()
print(content)

In [ ]:
#@title Compile Pycopy port for ESP32
! cd $dir_esp32 && source $git_espidf/export.sh && cd $dir_esp32 && make submodules && make
